In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Timing Systems under MPI

There is another way to view timings for methods called on specific group and component instances in an OpenMDAO model.  Not only does it
have lower overhead than the previously discussed instance-based profiler, but it also displays, when running your model under MPI, how that model spends its time across MPI processes.

The simplest way to use it is via the command line using the `openmdao timing` command. For example:
``` bash
    openmdao timing <your_python_script_here>
```

This will collect the timing data and pop up a table viewer like the one shown below.

![An example of a timing viewer](timing_viewer.png)


There is a row in the table for each method specified for each group or component instance in the model, on each MPI rank.  The default set of methods is `_solve_nonlinear`, `_solve_linear`, `_apply_nonlinear`, `_apply_linear`, and `_linearize`. There are columns in the table for mpi rank, number of procs, level in the system tree, whether a system is a parallel group or not, problem name, system path, method name, number of calls, total time, percent of total time, average time, min time, and max time. All columns are sortable, and are also filterable in most cases.

By using filters on various columns, it's a simple process to zero in on any level of the system tree containing children of a ParallelGroup in order to see how processes and execution time is split up among those children. For example, in the table shown above, we've filtered out all but tree level 3 and all systems but `traj.phases` and below.  Finally, we've filtered out all functions but `_solve_nonlinear`. This combination of filters shows us only the direct children of `traj.phases` which happens to be a ParallelGroup.  And we can see from the table that `traj.phases.climb2` is using 3 processes, while all of the other children share a single process.  The model in this example was run using 4 processes.  Looking at the time taken by `climb2` vs the other children, it might actually be better to run this model without using a ParallelGroup for `traj.phases` at all!


Documentation of options for all commands described here can be obtained by running the command followed by the -h option. For example:

``` bash
    openmdao timing -h
```

```
usage: openmdao timing [-h] [-o OUTFILE] [-f FUNCS] [-v VIEW] [--use_context] file

positional arguments:
  file                  Python file containing the model, or pickle file containing previously
                        recorded timing data.

optional arguments:
  -h, --help            show this help message and exit
  -o OUTFILE            Name of output file where timing data will be
                        stored. By default it goes to "timings.pkl".
  -f FUNCS, --func FUNCS
                        Time a specified function. Can be applied multiple times to specify multiple functions. Default methods are
                        ['_apply_linear', '_apply_nonlinear', '_linearize', '_solve_linear', '_solve_nonlinear'].
  -v VIEW, --view VIEW  View the output. Default view is "browser".
                        Other options are "text" for ascii output or
                        "none" for no output.
  --use_context         If set, timing will only be active within a
                        timing_context.

```

If you don't like the default set of methods, you can specify your own using the `-f` or `--funct` options.  This option can be applied multiple times to specify multiple functions.

The `-v` and `--view` options default to "browser", showing a table like the one above.  You can also choose "text" which will give you essentially an ascii dump of the table data, or "none" which generates no output other than a pickle file, typically "timings.pkl" that contains the timing data and can be used for later viewing.

The `--use_context` option is for occasions when you only want to time a certain portion of your script.  In that case, you can wrap the code of interest in a `timing_context` context manager as shown below.

In [ ]:
from openmdao.visualization.timing_viewer.timer import timing_context

# do some stuff that I don't want to time...

with timing_context():
    # do stuff I want to time
    pass

# do some other stuff that I don't want to time...

```{Warning}
If the `use_context` option is set, timing will not occur anywhere outside of `timing_context`, so be careful not to use that option on a script that doesn't use a `timing_context`, because in that case, no timing will be done.  
```

```{Warning}
If you *don't* specify `use_context` but your script *does* contain a `timing_context`, then that `timing_context` will be ignored and timing info will be collected for the entire script anyway.
```

```{Warning}
If your script exits with a nonzero exit code, the timing data will not be saved to a file.
```

After your script is finished running, you should see a new file called *timings.pkl* in your current directory. This is a pickle file containing all of the timing data.  If you didn't specify the `-v` or `--view` options, or if you specified a view option of "browser", you'll also see a file called *timing_report.html* which can be opened in a browser to view the interactive timings table discussed earlier.
